<a href="https://colab.research.google.com/github/AshOne91/PLAYDATA-COLAB/blob/main/dacon_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 적절한 전처리
# 피처엔지니어링
# 모델 변경
# 튜

,설립연도
0,2009
1,2023
2,2018
3,2016
4,2020
...,...
4371,2021
4372,2020
4373,2023
4374,2001


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/dacon_03/open.zip -d data

Archive:  /content/drive/MyDrive/dacon_03/open.zip
  inflating: data/sample_submission.csv  
  inflating: data/test.csv           
  inflating: data/train.csv          


In [ ]:
import pandas as pd
df = pd.read_csv('data/train.csv')

In [ ]:
df.describe(include='all')

,ID,설립연도,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),성공확률
count,4376,4376.000000,4376,3519,4376,4202.000000,4376,4376,3056.000000,4376.000000,4376.000000,4376.000000,3156,4376.000000
unique,4376,NaN,10,10,5,NaN,2,2,NaN,NaN,NaN,NaN,5,NaN
top,TRAIN_4375,NaN,CT001,에너지,IPO,NaN,Yes,No,NaN,NaN,NaN,NaN,4500-6000,NaN
freq,1,NaN,476,396,938,NaN,2257,2263,NaN,NaN,NaN,NaN,679,NaN
mean,NaN,2012.006856,NaN,NaN,NaN,2471.324369,NaN,NaN,49.214332,3314.568556,6452.202239,2.578103,NaN,0.537340
std,NaN,6.554872,NaN,NaN,NaN,1446.064215,NaN,NaN,27.141242,1880.103831,3782.792656,1.414590,NaN,0.241698
min,NaN,2001.000000,NaN,NaN,NaN,5.000000,NaN,NaN,1.000000,12.000000,12.000000,0.000000,NaN,0.100000
25%,NaN,2006.000000,NaN,NaN,NaN,1218.250000,NaN,NaN,27.000000,1651.500000,3094.250000,1.320000,NaN,0.400000
50%,NaN,2012.000000,NaN,NaN,NaN,2472.000000,NaN,NaN,49.000000,3398.000000,6478.500000,2.835000,NaN,0.600000
75%,NaN,2018.000000,NaN,NaN,NaN,3727.500000,NaN,NaN,72.000000,4908.250000,9692.500000,4.000000,NaN,0.700000


In [ ]:
!pip install pytorch-tabnet -q

In [ ]:
!pip install optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 13.0 MB/s eta 0:00:00


In [ ]:
%pip install pytorch-tabnet -q

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

import torch
from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.tab_model import TabNetRegressor

In [ ]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')

In [ ]:
#특성과 타겟 변수 분리
train = train.drop(columns=['ID'], axis = 1)
test = test.drop(columns=['ID'], axis = 1)

In [ ]:
# 범주형 데이터와 수치형 데이터를 분리 해서
numeric_cols = train.select_dtypes(exclude='object')
categorical_cols = train.select_dtypes(include='object')
numeric_cols.describe()

,설립연도,직원 수,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),성공확률
count,4376.000000,4202.000000,3056.000000,4376.000000,4376.000000,4376.000000,4376.000000
mean,2012.006856,2471.324369,49.214332,3314.568556,6452.202239,2.578103,0.537340
std,6.554872,1446.064215,27.141242,1880.103831,3782.792656,1.414590,0.241698
min,2001.000000,5.000000,1.000000,12.000000,12.000000,0.000000,0.100000
25%,2006.000000,1218.250000,27.000000,1651.500000,3094.250000,1.320000,0.400000
50%,2012.000000,2472.000000,49.000000,3398.000000,6478.500000,2.835000,0.600000
75%,2018.000000,3727.500000,72.000000,4908.250000,9692.500000,4.000000,0.700000
max,2023.000000,4998.000000,99.000000,7059.000000,14822.000000,5.000000,0.900000


In [ ]:
# 설립연도 타입 변환 (int -> object)
train['설립연도'] =train['설립연도'].astype('object')
test['설립연도'] =test['설립연도'].astype('object')

category_features = ['설립연도','국가','분야','투자단계','기업가치(백억원)']
numeric_features = ['직원 수','고객수(백만명)','총 투자금(억원)','연매출(억원)','SNS 팔로워 수(백만명)']
bool_features = ['인수여부','상장여부']

# LabelEncoder 객체를 각 범주형 feature별로 따로 저장하여 사용
encoders = {}

# 범주형 데이터를 encoding
for feature in category_features:
    encoders[feature] = LabelEncoder()
    train[feature] = train[feature].fillna('Missing')
    test[feature] = test[feature].fillna('Missing')
    train[feature] = encoders[feature].fit_transform(train[feature])
    test[feature] = encoders[feature].transform(test[feature])

# 불리언 값을 0과 1로 변환 ('Yes' → 1, 'No' → 0 으로 변환)
bool_map = {'Yes': 1, 'No': 0}

for feature in bool_features:
    train[feature] = train[feature].map(bool_map)
    test[feature] = test[feature].map(bool_map)

# 수치형 변수 결측치를 평균값으로 대체
for feature in numeric_features:
    mean_value = train[feature].mean()
    train[feature] = train[feature].fillna(mean_value)
    test[feature] = test[feature].fillna(mean_value)

# TabNet용 범주형 변수 인덱스(cat_idxs) 및 차원(cat_dims) 설정
features = [col for col in train.columns if col != '성공확률']
cat_idxs = [features.index(col) for col in category_features]
cat_dims = [train[col].max() + 1 for col in category_features]

<ipython-input-45-d9229aa851d1>:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train[feature] = train[feature].fillna('Missing')
<ipython-input-45-d9229aa851d1>:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test[feature] = test[feature].fillna('Missing')


In [ ]:
# 데이터 증강  Data Agumentation
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

def augment_numerical_data(data, column_name, num_samples):
    """
    수치형 데이터 증강 함수
    """
    mean = data[column_name].mean()
    std = data[column_name].std()
    min_val = data[column_name].min()
    max_val = data[column_name].max()

    # 정규분포를 따르는 새로운 데이터 생성
    new_data = np.random.normal(mean, std, num_samples)

    # 최소값과 최대값 범위 내로 제한
    new_data = np.clip(new_data, min_val, max_val)

    return new_data

def augment_categorical_data(data, column_name, num_samples):
    """
    범주형 데이터 증강 함수
    """
    # 각 범주의 빈도 계산
    value_counts = data[column_name].value_counts(normalize=True)

    # 빈도에 따라 새로운 데이터 생성
    new_data = np.random.choice(
        value_counts.index,
        size=num_samples,
        p=value_counts.values
    )

    return new_data

def augment_data(data, num_samples=1000):
    """
    전체 데이터 증강 함수
    """
    augmented_data = pd.DataFrame()

    # 수치형 변수 증강
    numerical_columns = ['설립연도', '직원 수', '고객수(백만명)',
                        '총 투자금(억원)', '연매출(억원)',
                        'SNS 팔로워 수(백만명)', '기업가치(백억원)']

    for col in numerical_columns:
        augmented_data[col] = augment_numerical_data(data, col, num_samples)

    # 범주형 변수 증강
    categorical_columns = ['국가', '분야', '투자단계', '인수여부', '상장여부']

    for col in categorical_columns:
        augmented_data[col] = augment_categorical_data(data, col, num_samples)

    # 성공확률 증강 (0.1에서 0.9 사이의 값으로 제한)
    augmented_data['성공확률'] = np.random.uniform(0.1, 0.9, num_samples)

    return augmented_data

# 데이터 증강
augmented_data = augment_data(train, num_samples=1000)

train = pd.concat([train, augmented_data])

In [ ]:
# 타겟 지정
target = train['성공확률']
X = train[features]
y = target

# KFold 설정
N_FOLDS = 5
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

models = [] # 모델 저장 리스트
cv_scores = []

for fold, (train_idx, valid_idx) in enumerate(kf.split(X)):
    print(f"\n🔁 Fold {fold+1}/{N_FOLDS}")

    X_train = X.iloc[train_idx].values
    y_train = y.iloc[train_idx].values.reshape(-1, 1)

    X_valid = X.iloc[valid_idx].values
    y_valid = y.iloc[valid_idx].values.reshape(-1, 1)

    # 비지도 사전학습
    print("▶ Pretraining...")

    pretrainer = TabNetPretrainer(
        cat_idxs=cat_idxs,
        cat_dims=cat_dims,
        seed=42,
        verbose=0
    )

    pretrainer.fit(
        X_train=X_train,
        max_epochs=100,
        batch_size=512,
        virtual_batch_size=64
    )

    # 지도 학습
    print("▶ Fine-tuning...")
    model = TabNetRegressor(
        cat_idxs=cat_idxs,
        cat_dims=cat_dims,
        seed=42,
        verbose=0,
        optimizer_fn=torch.optim.AdamW
    )

    model.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_valid, y_valid)],
        from_unsupervised=pretrainer,
        eval_metric=['mae'],
        max_epochs=100,
        patience=10
    )

    # 모델을 메모리에 저장
    models.append(model)
    cv_scores.append(model.best_cost)

print("\n✅ 모든 fold 모델 학습 완료!")


🔁 Fold 1/5
▶ Pretraining...


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


▶ Fine-tuning...


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:248: UserWarning: Loading weights from unsupervised pretraining
  warnings.warn("Loading weights from unsupervised pretraining")



Early stopping occurred at epoch 42 with best_epoch = 32 and best_val_0_mae = 0.20807

🔁 Fold 2/5
▶ Pretraining...


...

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


▶ Fine-tuning...


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:248: UserWarning: Loading weights from unsupervised pretraining
  warnings.warn("Loading weights from unsupervised pretraining")



Early stopping occurred at epoch 33 with best_epoch = 23 and best_val_0_mae = 0.20617

✅ 모든 fold 모델 학습 완료!


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
# 저장된 모델들로 예측
predictions_list = []

for fold, model in enumerate(models):
    print(f"Predict with fold {fold+1}")
    preds = model.predict(test[features].values)
    predictions_list.append(preds)

# 평균 예측
final_predictions = np.mean(predictions_list, axis=0)

Predict with fold 1
Predict with fold 2
Predict with fold 3
Predict with fold 4
Predict with fold 5


In [ ]:
sample_submission['성공확률'] = final_predictions
sample_submission.to_csv('./baseline_submission.csv', index = False, encoding = 'utf-8-sig')